In [1]:
import openai
import json
import os
import dotenv
import json


In [2]:
dotenv.load_dotenv(".env", override=True)
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
assert OPENAI_API_KEY is not None
openai.api_key = OPENAI_API_KEY

In [3]:
# Pricing library

# Test implementation
def getproductprice(productcode):
    # Your implementation here
    return "{'price': 199.99}"

# Define the function descriptions for the API
pricing_functions = [
    {
        "name": "getproductprice",
        "description": "Get the price for a product",
        "parameters": {
            "type": "object",
            "properties": {
                "productcode": {"type": "string",
                                "description": "The code of the product being ordered"}
            },
            "required": ["productcode"],
            "output": {
                "type": "object",
                "properties": {
                    "price": {"type": "number", "format": "currency"}
                }
            }
        }
    }
]

# Map the function names to the functions
pricing_function_map = {
    "getproductprice": getproductprice,
}

In [5]:
# Product library

# Test implementation
def getproductdescription(productcode):
    # Your implementation here
    return "{'description': 'My test product, pk of 5'}"

# Define the function descriptions for the API
product_functions = [
    {
        "name": "getproductdescription",
        "description": "Get the product description for a product",
        "parameters": {
            "type": "object",
            "properties": {
                "productcode": {"type": "string",
                                "description": "The code of the product"}
            },
            "required": ["productcode"],
            "output": {
                "type": "object",
                "properties": {
                    "description": {"type": "string"}
                }
            }
        }
    }
]

# Map the function names to the functions
product_function_map = {
    "getproductdescription": getproductdescription,
}

In [6]:
# Customer library

# Test implementation
def getcustomername(accountnumber):
    # Your implementation here
    return "{'name': 'Billy's Name'}"

# Define the function descriptions for the API
customer_functions = [
    {
        "name": "getcustomername",
        "description": "Get the name of a customer",
        "parameters": {
            "type": "object",
            "properties": {
                "accountnumber": {"type": "string",
                                "description": "The account number"}
            },
            "required": ["accountnumber"],
            "output": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"}
                }
            }
        }
    }
]

# Map the function names to the functions
customer_function_map = {
    "getcustomername": getcustomername,
}

In [7]:
# Availability Library


# Test Implementation
def getproductavailabilitydate(productcode):
    # Your implementation here
    return "{'availability_date': '2023-07-20'}"

def getproductinventorybyloc(loc):
    # Your implementation here
    return "{'inventory_details': [{'loc_id': 1, 'inventory': 100, 'availability': 80},\
        {'loc_id': 2, 'inventory': 200, 'availability': 180}]}"


# Define the function descriptions for the API
availability_functions = [
    {
        "name": "getproductavailabilitydate",
        "description": "Get the availability date for a product",
        "parameters": {
            "type": "object",
            "properties": {

                "productcode": {"type": "string", 
                                "description": "The code of the product being ordered"}
            },
            "required": ["productcode"],
            "output": {
                "type": "object",
                "properties": {
                    "availability_date": {"type": "string", "format": "date"}
                }
            }
        },
    },
    {
        "name": "getproductinventorybyloc",
        "description": "Get the inventory for a product by location",
        "parameters": {
            "type": "object",
            "properties": {
                "loc": {"type": "string",
                        "description": "The location of the inventory to check"}
            },
            "required": ["loc"],
            "output": {
                "type": "object",
                "properties": {
                    "inventory_details": {"type": "array",
                                        "items": {
                                                    "type": "object",
                                                    "properties": {
                                                        "loc_id": {"type": "integer"},
                                                        "inventory": {"type": "integer"},
                                                        "availability": {"type": "integer"}
                                                    },
                                        },
                                        "required": ["loc_id", "inventory", "availability"]
                    }
                }
            }
        }
    }
]

# Map the function names to the functions
availability_function_map = {
    "getproductavailabilitydate": getproductavailabilitydate,
    "getproductinventorybyloc": getproductinventorybyloc,
    "getproductprice": getproductprice,
}


In [8]:
# Mapping the library's
libraries = ["availability", "pricing", "product", "shipping"]   

library_functions = {
    "availability": availability_functions,
    "pricing": pricing_functions,
    "product": product_functions,
    "customer": customer_functions,
}

library_function_maps = {
    "avalability": availability_function_map,
    "pricing": pricing_function_map,
    "product": product_function_map,
    "customer": customer_function_map,
}

In [9]:
function = [{        "name": "determinelibrary",
        "description": "Returns a list of subjects that we should use to handle the user's question. The list is ordered by the confidence of the model in the subject. The choice of subjects is '{libraries}'.",
        "parameters": {
            "type": "object",
            "output": {
                "type": "object",
                "properties": {
                    "subjects": {"type": "array",
                                        "subject": {
                                                    "type": "object",
                                                    "properties": {
                                                        "library": {"type": "string"},
                                                        "enum": libraries
                                                    },
                                        },
                                        "required": ["library"]
                    }
                }
            }
        }
    }
]

def get_function_library(user_question):
  system_message = "You are going to read the users question and determine which library of functions or subjects should be used to handle the query."
  prompt = f"The user's question is: '{user_question}'. Which are the list of subjects are in this question?"
  messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]

  response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=function,  
    function_call="auto",
  )

  # Extracting the response from the model
  library_category = response.choices[0].text.strip()
  print(f"The library category is: '{library_category}'")
  # Validate and return the category
  valid_categories = library_functions
  if library_category.lower() in valid_categories:
      return library_category.lower()
  else:
      raise ValueError("Unexpected category received from the language model.")
 

In [10]:
# Description: This is the main file for the chatbot. It contains the conversation loop and the function calling.
# Assumes global variables: functions, available_functions

def run_conversation(system_message="", user_question=""):

    # Parse the messages to get the functions and available functions for the next question
    lbr = get_function_library(user_question)
    print(lbr)
    functions = library_functions[lbr]
    print(functions)
    available_functions = library_function_maps[lbr]
    print(available_functions)

    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_question}]

    i = 0

    while True:

        i = i + 1
        # print the current message
        print("Message " + str(i))
        print(messages[-1]["content"])

        
        # Call the model
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            functions=functions,  
            function_call="auto",
        )

        response_message = response["choices"][0]["message"]
        # print the response
        print(response_message["content"])
        print("---------------")

        # Check if GPT wanted to call a function
        if response_message.get("function_call"):
            function_name = response_message["function_call"]["name"]
            function_to_call = available_functions[function_name]  
            function_args = json.loads(response_message["function_call"]["arguments"])
            function_response = function_to_call(**function_args)

            # Send the info on the function call and function response to GPT
            messages.append(response_message)
            messages.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                }
            )
        else:
            print("Conversation complete.")
            break

    return response_message["content"]
    


In [11]:
# Run the conversation

# Define the system message
system_message = "You are a helpful Customer Service chatbot answering questions about products, especially delivery times, product availability, and product price."

# Start the conversation with the user's question
user_question = "What is the price of 123456, is it available, and how many do we have in location 2?" #input("Ask your question: ")

print("---------------")
print(run_conversation(system_message, user_question))

---------------


TypeError: __str__ returned non-string (type list)